In [1]:
!pip install -q transformers[torch] datasets[audio] accelerate evaluate jiwer librosa torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 101.2 MB/s eta 0:00:00


Load G drive and change directory

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/sinhala_pronunciation/

/content/drive/MyDrive/sinhala_pronunciation


Dataset Loader

In [4]:
# dataset_loader.py
import os, json, torch, torchaudio

root = "/content/drive/MyDrive/sinhala_pronunciation/dataset"

import os, json, torch, torchaudio

class SinhalaDataset(torch.utils.data.Dataset):
    def __init__(self, root):
        self.samples = []
        with open(f"{root}/phonemes.json", "r", encoding="utf-8") as f:
            self.data_map = json.load(f)

        for folder_name, info in self.data_map.items():
            folder_path = os.path.join(root, folder_name)

            if os.path.exists(folder_path):
                for f in os.listdir(folder_path):
                    if f.endswith(".wav"):
                        # Store the path AND the specific phoneme list
                        self.samples.append((os.path.join(folder_path, f), info["phonemes"]))
            else:
                print(f"⚠️ Missing folder: {folder_path}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        wav_path, phonemes = self.samples[idx]
        wav, sr = torchaudio.load(wav_path)
        if sr != 16000:
            wav = torchaudio.functional.resample(wav, sr, 16000)

        return wav.squeeze(), phonemes


Test data loading

In [5]:
dataset = SinhalaDataset(root)
print(len(dataset))

wav, phones = dataset[0]
print(wav.shape)
print(phones)


1618
torch.Size([10664])
['b', 'a', 'l', 'l', 'a:']


Updating vocabulary with mappings for silence, etc

In [6]:
import json

# existing vocab
existing_vocab = {
    "a": 0, "a:": 1, "b": 2, "l": 3, "i": 4, "d": 5, "th": 6,
    "p": 7, "o": 8, "k": 9, "s": 10, "m": 11, "n": 12, "y": 13,
    "u": 14, "r": 15, "w": 16, "h": 17, "g": 18, "i:": 19, "sh": 20
}

training_vocab = existing_vocab.copy()
training_vocab["[PAD]"] = 21
training_vocab["[UNK]"] = 22
# training_vocab["[BLANK]"] = 23 # separate sounds

with open("vocab.json", "w", encoding="utf-8") as f:
    json.dump(training_vocab, f)

print(f"✅ Created training vocab with {len(training_vocab)} tokens.")

✅ Created training vocab with 23 tokens.


Training execution

In [31]:
import torch
import json
import os
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2ForCTC

# --- 1. SET DEVICE ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 2. LOAD VOCAB & PROCESSOR ---
with open("vocab.json", "r", encoding="utf-8") as f:
    training_vocab = json.load(f)

tokenizer = Wav2Vec2CTCTokenizer(
    "./vocab.json",
    unk_token="[UNK]",
    pad_token="[PAD]",
    word_delimiter_token=None
)
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0,
    do_normalize=True, return_attention_mask=True
)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

from transformers import Wav2Vec2Config

# --- 3. LOAD MODEL ---
config = Wav2Vec2Config.from_pretrained("facebook/wav2vec2-xls-r-300m")

# 2. Update config with specific parameters
config.ctc_loss_reduction = "mean"
config.pad_token_id = processor.tokenizer.pad_token_id
config.vocab_size = len(processor.tokenizer)
# config.ctc_blank_id = training_vocab.get("[BLANK]", 0)

# 3. Load the model using this config
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    config=config,
    ignore_mismatched_sizes=True # Required because, changed vocab_size
)

model.to(device)
model.freeze_feature_encoder()

# --- 4. DATA COLLATOR ---
def collate_fn(batch):
    # Separate the waveforms and the phoneme lists
    audio_list = [item[0].numpy() for item in batch]
    phoneme_lists = [item[1] for item in batch]

    # 1. Process Audio (Input)
    inputs = processor(
        audio_list,
        sampling_rate=16000,
        return_tensors="pt",
        padding=True
    )

    # 2. Process Labels (Target) using the tokenizer directly
    # This avoids the 'multiple values for padding' error
    labels_batch = processor.tokenizer(
        phoneme_lists,
        is_split_into_words=True,
        padding=True,
        return_tensors="pt"
    )

    # Replace padding index with -100 so CTC loss ignores it
    labels = labels_batch.input_ids.masked_fill(
        labels_batch.input_ids == processor.tokenizer.pad_token_id, -100
    )

    return inputs.input_values.to(device), labels.to(device)

# --- 5. STABILIZED TRAINING LOOP ---
from torch.nn.utils import clip_grad_norm_

# 1. Update Config to handle infinite loss
model.config.ctc_zero_infinity = True

loader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
optimizer = AdamW(model.parameters(), lr=1e-4)

model.train()
print("🚀 Starting stabilized training...")

for epoch in range(20): # used 15, 18

    if epoch == 5:
      model.unfreeze_feature_encoder()
      print("🔓 Feature encoder unfrozen")

    epoch_loss = 0
    for batch_idx, (inputs, labels) in enumerate(loader):
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_values=inputs, labels=labels)
        loss = outputs.loss

        # 2. Check for NaN before backprop
        if torch.isnan(loss):
            print(f"⚠️ Skip NaN batch {batch_idx}")
            continue

        loss.backward()

        # 3. Gradient Clipping (Crucial for stability)
        clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        epoch_loss += loss.item()

        if batch_idx % 10 == 0:
            print(f"Batch {batch_idx} | Loss: {loss.item():.4f}")

    avg_loss = epoch_loss / len(loader)
    print(f"✅ Epoch {epoch} | Avg Loss: {avg_loss:.4f}")

# --- 6. SAVE ---
model.save_pretrained("sinhala-pronunciation-model")
processor.save_pretrained("sinhala-pronunciation-model")
print("💾 Done! Model saved to 'sinhala-pronunciation-model' folder.")

Using device: cuda


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🚀 Starting stabilized training...
Batch 0 | Loss: 5.1879
Batch 10 | Loss: 3.3018
Batch 20 | Loss: 3.6023
Batch 30 | Loss: 2.6611
Batch 40 | Loss: 1.9498
Batch 50 | Loss: 1.7895
Batch 60 | Loss: 1.7031
Batch 70 | Loss: 1.7950
Batch 80 | Loss: 2.2586
Batch 90 | Loss: 1.8603
Batch 100 | Loss: 2.3659
Batch 110 | Loss: 1.7227
Batch 120 | Loss: 1.7723
Batch 130 | Loss: 1.7000
Batch 140 | Loss: 1.6639
Batch 150 | Loss: 2.4130
Batch 160 | Loss: 1.6893
Batch 170 | Loss: 1.6230
Batch 180 | Loss: 2.0014
Batch 190 | Loss: 1.6639
Batch 200 | Loss: 1.4880
Batch 210 | Loss: 2.0273
Batch 220 | Loss: 1.8302
Batch 230 | Loss: 1.9126
Batch 240 | Loss: 1.7736
Batch 250 | Loss: 1.5837
Batch 260 | Loss: 1.7097
Batch 270 | Loss: 1.5888
Batch 280 | Loss: 1.9160
Batch 290 | Loss: 1.6132
Batch 300 | Loss: 1.6456
Batch 310 | Loss: 1.6714
Batch 320 | Loss: 1.8384
Batch 330 | Loss: 1.6013
Batch 340 | Loss: 2.0988
Batch 350 | Loss: 1.6631
Batch 360 | Loss: 1.6865
Batch 370 | Loss: 1.8957
Batch 380 | Loss: 1.8457
Ba

AttributeError: 'Wav2Vec2ForCTC' object has no attribute 'unfreeze_feature_encoder'

Scoring engine

Attemp #1

In [8]:
# def evaluate_pronunciation(wav_path, target_phonemes):
#     model.eval()
#     wav, sr = torchaudio.load(wav_path)
#     if sr != 16000:
#         wav = torchaudio.functional.resample(wav, sr, 16000)

#     input_values = processor(wav.squeeze().numpy(), sampling_rate=16000, return_tensors="pt").input_values.to(device)

#     with torch.no_grad():
#         logits = model(input_values).logits
#         probs = torch.softmax(logits, dim=-1)

#     phoneme_results = []
#     for p in target_phonemes:
#         p_id = training_vocab.get(p)
#         if p_id is not None:
#             # We look for the HIGHEST confidence the model had for this sound
#             # anywhere in the audio clip.
#             p_score = torch.max(probs[0, :, p_id]).item()

#             # Boost logic: If the model is 70% sure, for a child, that's an A!
#             # We scale the score slightly to be more encouraging.
#             adjusted_score = min(1.0, p_score * 1.2)

#             phoneme_results.append({
#                 "phoneme": p,
#                 "score": adjusted_score,
#                 "feedback": get_sinhala_feedback(adjusted_score)
#             })
#     return phoneme_results

Attempt #2

In [15]:
import torchaudio.functional as F

def evaluate_pronunciation(wav_path, target_phonemes):
    model.eval()

    wav, sr = torchaudio.load(wav_path)
    if sr != 16000:
        wav = torchaudio.functional.resample(wav, sr, 16000)

    inputs = processor(
        wav.squeeze().numpy(),
        sampling_rate=16000,
        return_tensors="pt"
    ).input_values.to(device)

    with torch.no_grad():
        emissions = model(inputs).logits.log_softmax(-1)

    target_ids = torch.tensor(
        [training_vocab[p] for p in target_phonemes],
        device=device
    ).unsqueeze(0)

    alignment, scores = F.forced_align(
        emissions,
        target_ids,
        blank=processor.tokenizer.pad_token_id
    )

    results = []
    for i, p in enumerate(target_phonemes):
        raw_gop = scores[0][i].item()
        confidence = normalize_gop(raw_gop)

        results.append({
            "phoneme": p,
            "score": confidence,
            "feedback": get_sinhala_feedback(confidence)
        })

    return results


In [16]:
# Normalize GOP

import math

def normalize_gop(log_prob, min_lp=-15.0, max_lp=-2.0):
    """
    Maps log-probability to [0,1] confidence
    """
    log_prob = max(min(log_prob, max_lp), min_lp)
    return (log_prob - min_lp) / (max_lp - min_lp)


Sinhala Feedback

In [10]:
def get_sinhala_feedback(score):
    if score >= 0.85:
        return "ඉතා විශිෂ්ටයි! ශබ්දය ඉතා නිවැරදිව උච්චාරණය කළා. 🌟" # Excellent
    elif score >= 0.65:
        return "හොඳයි, නමුත් තව ටිකක් පුහුණු වෙමු. ශබ්දය තව පැහැදිලි කරන්න උත්සාහ කරන්න. 👍" # Good, try more
    elif score >= 0.40:
        return "නැවත උත්සාහ කරමු. දිව සහ තොල් පිහිටීම ගැන සැලකිලිමත් වන්න. 👂" # Try again, watch tongue/lips
    else:
        return "කණගාටුයි, ශබ්දය හඳුනා ගැනීමට අපහසුයි. නැවත පැහැදිලිව පවසන්න. ❌" # Hard to recognize

Testing

In [29]:
test_file = "/content/drive/MyDrive/sinhala_pronunciation/dataset/balla/sound-72.wav"
test_targets = ["b", "a", "l", "l", "a:"]

results = evaluate_pronunciation(test_file, test_targets)

print(f"--- උච්චාරණ වාර්තාව (Pronunciation Report) ---")
for r in results:
    print(f"Phoneme: {r['phoneme']} | Score: {r['score']:.2f}")
    print(f"Feedback: {r['feedback']}")
    print("-" * 30)

--- උච්චාරණ වාර්තාව (Pronunciation Report) ---
Phoneme: b | Score: 0.23
Feedback: කණගාටුයි, ශබ්දය හඳුනා ගැනීමට අපහසුයි. නැවත පැහැදිලිව පවසන්න. ❌
------------------------------
Phoneme: a | Score: 0.23
Feedback: කණගාටුයි, ශබ්දය හඳුනා ගැනීමට අපහසුයි. නැවත පැහැදිලිව පවසන්න. ❌
------------------------------
Phoneme: l | Score: 0.28
Feedback: කණගාටුයි, ශබ්දය හඳුනා ගැනීමට අපහසුයි. නැවත පැහැදිලිව පවසන්න. ❌
------------------------------
Phoneme: l | Score: 0.31
Feedback: කණගාටුයි, ශබ්දය හඳුනා ගැනීමට අපහසුයි. නැවත පැහැදිලිව පවසන්න. ❌
------------------------------
Phoneme: a: | Score: 0.32
Feedback: කණගාටුයි, ශබ්දය හඳුනා ගැනීමට අපහසුයි. නැවත පැහැදිලිව පවසන්න. ❌
------------------------------


/tmp/ipython-input-3860509179.py:24: UserWarning: torchaudio.functional._alignment.forced_align has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  alignment, scores = F.forced_align(
